In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Outline of this notebook:
1. [Exploratory Data Analysis](#Exploratory-Data-Analysis) (EDA): This where we explore and analyze the dataset to summarize thier main characteristics : 
 * [Visualization](#Visualization) 
 * [Correlation Matrix](#Correlation-Matrix)
1. [Data Processing](#Data-Processing): This is where we manipulate the explored and analyzed data to covert into meaningful information that can be used my our estimators:  
 * [Missing Data](#Missing-Data)
 * [Outliers](#Outliers)
 * [Adding New Features](#Adding-New-Features)
 * [Data Normalization](#Data-Normalization)
 * [Biased Features](#Biased-Features)
1. [Models Building](#Models-Building) (13 models): We would first train all of the models to search for models that has high cv score. These models are then ensemble together using [VotingRegressor](#VotingRegressor) to further increase it's performance
    * Ridge
    * Lasso
    * Elastic Net
    * RandomForestRegressor
    * ExtraTreeRegressor
    * BaggingRegressor
    * HuberRegressor
    * BayesianRidge
    * XGBRegressor
    * DecisionTreeRegressor
    * KNeighborsRegressor
    * GradientBoostingRegressor
 
 [Conclusion](#Conclusion)




In [ ]:
## Load necessary packages for EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # for data visualization use
import seaborn as sns # for data visualization use
import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore") # to filter out warnings messages

 ****Let's load out data****


In [ ]:
df =  pd.read_csv("../input/housesalesprediction/kc_house_data.csv")

## Take a quick pick on the data
print(df.info()) 
print("\n\n",df.nunique()) # Return the number of unique values in each feature
print("\n\n",df.head()) #Return the first 5 row of the dataframe


We can see that there are a total of 20 numerical features and 1 categorical feature in this dataset. The bathrooms feature has some float values for example 2.25 (float number). The dataset didn't not specify clearly on the meaning of the "0.25". Upon searching the web, I found out that:
* A full bathroom consists of : tub, sink, toilet and a separate shower.
* A three-quarter bathroom consists of : sink, shower and toilet.
* A half bathrooms consists of : a bathtub and a (sink or toilet).
* A quarter bathrooms consists of : a toilet or a shower stall, It is commonly found in basement of old houses.

So the "2.25" would indicates the house consists of two full bathrooms and a quarter bathroom

In [ ]:
## Drop the "id" column. It has nothing to do with the "price" column
df = df.drop("id",axis=1)


# Exploratory Data Analysis
## Visualization
Let's take a look on the target variable - "price"

In [ ]:
## Plot 1
plt.figure(figsize=(12,9))
plt.subplot(121)
plt.title("Price Distribution")
sns.distplot(df["price"])

## Plot 2
plt.subplot(122)
plt.scatter(range(df.shape[0]),np.sort(df["price"].values))
plt.title("Price Curve Distribution",fontsize=15)
plt.ylabel("Amount ($)",fontsize=12)

plt.show()

The price distibution has a positive skeness. This indicates the majority of the population has the ability to afford low to mid range house price while a few have the ability to afford high range house price. 

****Now let's have a look at the numerical data****

In [ ]:
num_cols = df.select_dtypes(exclude="object").columns.tolist() # Taking all numerical feature's name
num_cols.remove("price") # Remove the target variable 

for col in num_cols:
    sns.lmplot(data=df,x=col,y="price")
    plt.show()

As shown on the diagram, we can observe that:

1. bedrooms : some houses don't have this feature
1. bathrooms : some houses don't have this feature
1. sqft_living : has linear relationship with price
1. sqft_lot : no comment
1. waterfront : some houses don't have this feature
1. view : some houses don't have this feature & has linear relationship with price
1. condition : no comment
1. grade : has linear relationship with price
1. sqft_above : has a strong linear relationship with price
1. sqft_basement : some houses don't have this feature
1. yr_built : newer house has higher price value
1. yr_renovated : some houses have not been renovated
1. zipcode : no comment
1. lat : houses further to the north have higher price
1. long : houses further to the west have higher price
1. sqft_living15 : strong linear relationship with price
1. sqft_lot15 : no comment

Note: New columns can be created based on houses absent of certain feature. For instance, absent on yr_renovated feature,create a new column name "renovate" with value 1 indicates the house had been renovated or value 0 indicates house didn't renovate.

****Map visualization with folium****

In [ ]:
# Import packages
import folium
from folium.plugins import HeatMap

# Define a function to generate the basemap
def generateBaseMap(default_loc=[df.lat.loc[1],df.long.loc[1]]): #set the default location to the first house geographical position
    base_map = folium.Map(location=default_loc,control_scale=True)
    return base_map

#Generate the Heatmap
basemap = generateBaseMap()
#Add a weightage for heatmap density purpose
df_map = df.copy()
df_map["weightage"] = 1

# Add Icon to the option map
s = folium.FeatureGroup(name="icon").add_to(basemap)
# Add marker for the house with the highest price
max_price = df.loc[df.price.idxmax()]
folium.Marker([max_price["lat"],max_price["long"]],popup="Highest Price:${}".format(max_price.price),icon=folium.Icon(color="green")).add_to(s)

# Add heatmap
HeatMap(data=df_map[["lat","long","weightage"]].groupby(["lat","long"]).sum().reset_index().values.tolist(),radius=8,max_zoom=13,name="Heat Map").add_to(basemap)
folium.LayerControl(collapsed=False).add_to(basemap)

#Show the diagram
basemap


Warmer colors showing higher density of houses in that area. In contrast, colder colors showing low density of houses in that area

# Missing Data
In real life scenario, missing data is common issue to be deal with. 

In [ ]:
## Missing data
print(df.isnull().any()) # Return True if there a present of missing data

It seems that missing data is not an issue in this dataset.
Note: 
1. numerical features with missing data are replaced with the feature's median value
2. categorical features with missing are replaced with the feature's mode value

In [ ]:
## Lower Triangle Correlation Matrix
mask = np.zeros_like(df.corr(),dtype=bool)
mask[np.triu_indices_from(mask)] = True

## Plot Spearman Correlation Matrix
plt.figure(figsize=(12,9))
plt.title("Spearman Correlation Matrix",fontsize=16)
sns.heatmap(df.corr(),annot=True,fmt=".2f",mask=mask, square=True,linewidth=1.0,annot_kws={"fontsize":10})
plt.show()

## Plot top 10 most correlated features
feats = df.corr().nlargest(10,"price")["price"].index
cm = np.corrcoef(df[feats].values.T)
plt.figure(figsize=(12,9))
plt.title("Top 10 Correlated Features",fontsize=16)
sns.heatmap(cm,annot=True,fmt=".2f",linewidths=1.0,square=True,xticklabels=feats,yticklabels=feats,annot_kws={"fontsize":14})
plt.show()





# **Correlation Matrix**
Let's found out each feature correlation with each other and the target variable - price

From the correlation matrix, we can see that sqft_living, grade, sqft_above, sqft_living15 and bathrooms have ≥0.5 correlation with the target variable. These features have great influence of the regression model performance. However we can see that, sqft_living and sqft_above have a correlation of 0.88, we would pick the one that has higher correlate value with "price" - in this case sqft_living. Two features with ≥ 0.8 correlate value are just like a pair of twins, the characteristics are mostly similar therefore there's no need of these "twins feature". 

In [ ]:
## Drop sqft_above feature
df = df.drop("sqft_above",axis=1)

# Data Processing
## Missing Data
In real life scenario, missing data is common issue to be deal with.

In [ ]:
## Missing data
print(df.isnull().any()) # Return True if there's a present of missing data

## Outliners
An outlier is an observation point that is distant from other observation.


In [ ]:
from sklearn.preprocessing import StandardScaler

price_scaled = StandardScaler().fit_transform(df["price"][:,np.newaxis]) #scale the price
high_range = price_scaled[price_scaled[:,0].argsort()][-10:]
low_range = price_scaled[price_scaled[:,0].argsort()][:10]

print("Outer range (low) of the distribution :\n{}".format(low_range))
print("Outer range (high) of the distribution :\n{}".format(high_range))

We can see that there are no outliers in the low range of the distribution, however there's three outliers on the high range of distribution. Remove them.


In [ ]:
#Remove the outliers
outliers = df["price"].sort_values(ascending=False)[:3].index 
df = df.drop(outliers,axis=0)


## Adding New Features
Refering to the lmplots, there are houses that do not have some features. We can add a new feature on these.



In [ ]:
## define a function
presence = lambda x: 1 if x > 0 else 0 #Return 1 if there the house has the feature or else 0 for absence of that feature
features = ["bedrooms","bathrooms","waterfront","view","sqft_basement","yr_renovated"]

for feat in features:
    df["Has_"+feat] = df[feat].transform(presence)

## Create new columns based on the date feature
df["date"] = pd.to_datetime(df["date"]) # Convert to datetime
yr = lambda x: x.year ; month = lambda x: x.month 

df["yr_sold"] = df["date"].transform(yr)
df["month_sold"] = df["date"].transform(month)

## Create a new column based the age renovation
df["age_rnv"] = df["yr_sold"] - df["yr_built"]
df.loc[df["Has_yr_renovated"] == 1,"age_rnv"] = df["yr_sold"] - df["yr_renovated"]

## Remove yr_renovated as we have age_rnv
df = df.drop("yr_renovated",axis=1)
df = df.drop("date",axis=1)

## Let's check on the all the features
print(df.head())

## Data Normalization
The goal of normalization is to change the values of numeric columns in the dataset to use a common scale, without distorting differences in the ranges of values or losing information

* Test for the null hypothesis that the data was drawn from a normal distribution using Shapiro-Wilk test (scipy.stats.shapiro()). If the p value is less than or equal than 0.05 (assuming a 95% confidence level), the data is not normally distibuted. We need to normaliza the data that are not drawn from normal distibution.
Visualization the feature's distibution before and after the normalization

* Skewness.
 * As a rule of thumb: skewness values greater than 1.0 or less than -1.0 is considered highly skewed and skewness values greater than 0.5 or less than -0.5 is considered moderatedly skewed.
* Kurtosis:
 * Kurtosis is a measure of whether the data are heavy-tailed or light-tailed relative to a normal distribution


In [ ]:
# Let's look at the Target Variable first
from scipy.stats import skew, kurtosis
from scipy.stats import shapiro

# Check the p value from the Shapiro-Wilk Test
stc , p_value = shapiro(df.price.values)
print("p value of price feature is {}".format(p_value))

****price feature is not drawn from a normal distribution as it has a value of 4.115807574271001e-36 which is less than 0.05.**** 

In [ ]:
# Comparing the feature's skewness & kurtosis before and after normalization through visualization
fig,ax = plt.subplots(1,2,figsize=(12,9))

#Before Normalization
skewness = format(skew(df.price),".2f") ; kurt = format(kurtosis(df.price),".2f")
sns.distplot(df["price"],ax=ax[0])
ax[0].legend(["Skewness: {}\nKurtosis: {}".format(skewness,kurt)],loc="upper_right") #Note using legend to plot the skewness and kurtosis is not a corrrect & professional way, I did it for the purpose of cleaner visualization. 
ax[0].set(title="Before Normalization")

#After Normalization
df["price"] = np.log1p(df["price"]) # Normaliza the "price" feature
skewness = format(skew(df.price),".2f") ; kurt = format(kurtosis(df.price),".2f")
sns.distplot(df["price"],ax=ax[1])
ax[1].legend(["Skewness: {}\nKurtosis: {}".format(skewness,kurt)],loc="upper_right")
ax[1].set(title="After Normalization")
plt.show()

Great! After normalization, the "price" feature is more normally distributed.



In [ ]:
# import packages
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p

# Now let's normalize continuous numerical predictors that are not drawn from a normal distribution.
cols = ["sqft_living","sqft_lot","sqft_basement","sqft_living15","sqft_lot15"]

norm_test = lambda x: shapiro(x)[1] < 0.05 # Check if the p value is less than 0.05
num_feats = df[cols].apply(norm_test)

for col in cols:
    fig, ax = plt.subplots(1,2)
    sns.distplot(df[col],ax=ax[0])
    skewness = format(skew(df[col]),".2f") ; kurt = format(kurtosis(df[col]),".2f")
    ax[0].legend(["Skewness: {}\nKurtosis: {}".format(skewness,kurt)],loc="upper_right")
    ax[0].set(title="Before Normalization")
    
    df[col] = boxcox1p(df[col],boxcox_normmax(df[col]+1)) #Normalize the feature
    sns.distplot(df[col],ax=ax[1])
    skewness = format(skew(df[col]),".2f") ; kurt = format(kurtosis(df[col]),".2f")
    ax[1].legend(["Skewness: {}\nKurtosis: {}".format(skewness,kurt)],loc="upper_right")
    ax[1].set(title="After Normalization")
    plt.show()

## Biased Features
**Remove features that are extreme bias or imbalanced**


In [ ]:
all_feats = df.columns.tolist()

#Create an empty datatframe
percent = pd.DataFrame(columns = ["frequency"],index = all_feats) 

# Calculate the percentage of the mode value in the feature
for col in all_feats:
    percent.loc[col,"frequency"] = df[col].value_counts().iloc[0]/df.shape[0]  * 100

bias_feats = percent[percent["frequency"] > 99.94].index.tolist() # Threshold of 99.94%

##Remove the biased features
df = df.drop(bias_feats,axis=1)

    

# Models Building
All data should be standardize so that all the features are on the ****same scale****.

In [ ]:
# import models 
from sklearn.linear_model import Ridge,Lasso,ElasticNet,HuberRegressor,BayesianRidge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, GradientBoostingRegressor,AdaBoostRegressor, VotingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

# import necessary packages for model building
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold,cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Split the data into train and test set
y = df.price.values ; X = df.drop("price",axis=1).values # target variable would be the price and the else would be the predictors

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=2)

# Create an empty list for pipeline 
pipeline_models = []

# Assign all models into the a list
seed = 2
models = [Ridge(tol=10,random_state=seed),
          Lasso(tol=1,random_state=seed),
          ElasticNet(random_state=seed),
          HuberRegressor(),
          BayesianRidge(),
          RandomForestRegressor(random_state=seed),
          ExtraTreesRegressor(random_state=seed),
          BaggingRegressor(random_state=seed),
          GradientBoostingRegressor(),
          XGBRegressor(),
          DecisionTreeRegressor(),
          KNeighborsRegressor(),
          AdaBoostRegressor(random_state=seed)]

model_names = ["Ridge","Lasso","Elastic","Hub_Reg","BayRidge","RFR","ETR","BR","GBoost_Reg","XGB_Reg","DT_Reg","KNN_Reg","Ada_Reg"] # All models' labels

# Assign each model to a pipeline
for name,model in zip(model_names,models):
    pipeline = ("Scaled_"+name,
                Pipeline([("Scaler",StandardScaler()),
                         (name,model)
                         ]))
    pipeline_models.append(pipeline)


## Train the models
Train all the models and evaluate each of thier performance using K-fold Cross Validation score.

In [ ]:
## Create a dataframe to store all the model's cross validation score
evaluate = pd.DataFrame(columns=["model","cv","std","cv_all"]) #note: "cv" for mean cv & "cv_all" for later visualization use

for name,model in pipeline_models:
    kfold = KFold(n_splits=7) # 7 times
    cv = cross_val_score(model,X_train,y_train,cv=kfold,n_jobs= -1,scoring="r2")
    
    row = evaluate.shape[0]
    evaluate.loc[row,"model"] = name
    evaluate.loc[row,"cv"] = cv.mean()
    evaluate.loc[row,"std"] = "+/- {}".format(cv.std())
    evaluate.loc[row,"cv_all"] = cv
    
evaluate = evaluate.sort_values("cv",ascending=False)


In [ ]:
## Visualization on the cv score

fig, ax = plt.subplots (1,2,figsize=(18,12))
b = sns.barplot(x=evaluate["model"],y=evaluate["cv"],ax=ax[0],palette = sns.cubehelix_palette(evaluate.shape[0]))
for rec in b.patches:
    height = rec.get_height()
    ax[0].text(rec.get_x()+rec.get_width()/2,height * 1.01, round(height,4),ha="center")
ax[0].set(title="All models' CV score")
ax[0].set_xticklabels(evaluate["model"].tolist(),rotation=90)

sns.boxplot(x=evaluate["model"].tolist(),y=evaluate["cv_all"].tolist(),ax=ax[1])
ax[1].set(title="All models' CV distribution scores")
ax[1].set_xticklabels(evaluate["model"].tolist(),rotation=90)
plt.show()

Great !!    XGB_Reg has the best cv score of 0.8997. Now let's use voting regressor to combine models that have cv score above 0.85. 

These models are:
* XGBRegressor
* ExtraTreesRegressor
* RandomForestRegressor
* GradientBoostingRegressor
* BaggingRegressor

## VotingRegressor
A voting regressor is an ensemble meta-estimator that fits several base regressors, each on the whole dataset. Then it averages the individual predictions to form a final prediction

****Note:**** VotingRegressor works well with models that are not identical, for example: XGBRegressor with ExtraTreesRegressor. ExtraTreesRegressor and RandomTreesRegressor are highly identifical so we would just pick the one with a higher cv score (in this case ExtraTreesRegressor)

In [ ]:
## Create list to store all the combinations
votings = []

# XGBRegressor only                                 
votings.append(("Scaled_XGBR",Pipeline([("Scaler",StandardScaler()),
                                        (("XGB_Reg",XGBRegressor()))
                                       ])))

# All models
votings.append(("Scaled_XGBR_ETR_GBoostR_BR",Pipeline([("Scaler",StandardScaler()),
                                                       ("Voting",VotingRegressor([("XGB_Reg",XGBRegressor()),
                                                                                  ("ETR",ExtraTreesRegressor(random_state=seed)),
                                                                                  ("GBoost_Reg",GradientBoostingRegressor()),
                                                                                  ("BR",BaggingRegressor())
                                                                                 ]))
                                                     ])))

# XGBR with ETR combinations               
votings.append(("Scaled_XGBR_ETR",Pipeline([("Scaler",StandardScaler()),
                                            ("Voting",VotingRegressor([("XGB_Reg",XGBRegressor()),
                                                                       ("ETR",ExtraTreesRegressor(random_state=seed))
                                                                      ]))
                                           ])))

#XGBR with ETR & GBoost_Reg combinations              
votings.append(("Scaled_XGBR_ETR_GBoostR",Pipeline([("Scaler",StandardScaler()),
                                                      ("Voting",VotingRegressor([("XGB_Reg",XGBRegressor()),
                                                                                 ("ETR",ExtraTreesRegressor(random_state=seed)),
                                                                                 ("GBoost_Reg",GradientBoostingRegressor())
                                                                                ]))
                                                   ])))

#XGBR with ETR & BR combinations
votings.append(("Scaled_XGBR_ETR_BR",Pipeline([("Scaler",StandardScaler()),
                                               ("Voting",VotingRegressor([("XGB_Reg",XGBRegressor()),
                                                                          ("ETR",ExtraTreesRegressor(random_state=seed)),
                                                                          ("BR",BaggingRegressor())
                                                                       ]))
                                             ])))
#XGBR with GBoost_Reg combinations              
votings.append(("Scaled_XGBR_GBoostR",Pipeline([("Scaler",StandardScaler()),
                                                ("Voting",VotingRegressor([("XGB_Reg",XGBRegressor()),
                                                                           ("GBoost_Reg",GradientBoostingRegressor())
                                                                          ]))
                                               ])))

#XGBR with GBoost_Reg & BR combinations              
votings.append(("Scaled_XGBR_GBoostR_BR",Pipeline([("Scaler",StandardScaler()),
                                                   ("Voting",VotingRegressor([("XGB_Reg",XGBRegressor()),
                                                                              ("GBoost_Reg",GradientBoostingRegressor()),
                                                                              ("BR",BaggingRegressor())
                                                                             ]))
                                                   ])))

#XGBR with BR combinations                                 
votings.append(("Scaled_XGBR_BR",Pipeline([("Scaler",StandardScaler()),
                                           ("Voting",VotingRegressor([("XGB_Reg",XGBRegressor()),
                                                                      ("BR",BaggingRegressor())
                                                                     ]))
                                           ])))

#Create an empty dataframe to store the best score
evaluate_vote = pd.DataFrame(columns=["model","cv","std"])

## train the model and assign the cv score to evaluate_vote
for name, model in votings:
    kfold = KFold(n_splits=7)
    cv = cross_val_score(model,X_train,y_train,scoring="r2",cv=kfold,n_jobs=-1)
    
    row = evaluate_vote.shape[0]
    evaluate_vote.loc[row,"model"] = name
    evaluate_vote.loc[row,"cv"] = cv.mean()
    evaluate_vote.loc[row,"std"] = "+/- {}".format(cv.std())
    
evaluate_vote = evaluate_vote.sort_values("cv",ascending=False)
## Check on the scores
print(evaluate_vote)                       

In [ ]:
# Visualization
fig, ax = plt.subplots (1,1,figsize=(18,12))
b = sns.barplot(x=evaluate_vote["model"],y=evaluate_vote["cv"],ax=ax,palette = sns.cubehelix_palette(evaluate.shape[0]))
for rec in b.patches:
    height = rec.get_height()
    ax.text(rec.get_x()+rec.get_width()/2,height * 1.01, round(height,4),ha="center")
ax.set(title="All Combinations Voting Regressor cv scores")
ax.set_xticklabels(evaluate_vote["model"].tolist(),rotation=90)
plt.show()

# Conclusion
The best model for this data would be Scaled Voting Regressor that fits XGBRegressor and ExtraTreesRegressor. It scored a cv value of 0.9035 . Lasso and Elastic Net, which are regularization regression algorithmns that are not suitable in this dataset.
It's been my pleasure to share my notebook with you. Do hit that "upvote" button if you enjoy reading it. Thank you !  
